# Hybrid Search with FastEmbed & Qdrant



## What will we do?
This notebook demonstrates the usage of Hybrid Search with FastEmbed & Qdrant. 

1. Setup: Download and install the required dependencies
2. Preview data: Load and preview the data
3. Create Sparse Embeddings: Create SPLADE++ embeddings for the data
4. Create Dense Embeddings: Create BGE-Base-en-v1.5 embeddings for the data
5. Indexing: Index the embeddings using Qdrant
6. Search: Perform Hybrid Search using FastEmbed & Qdrant

## Setup

In order to get started, you need only two dependencies, and we'll install them next:

In [1]:
# !pip install -qU qdrant-client fastembed datasets transformers

In [2]:
import json
from typing import List

import numpy as np
import pandas as pd
from datasets import load_dataset
from qdrant_client import QdrantClient
from qdrant_client.http.models import (
    Distance,
    NamedSparseVector,
    NamedVector,
    SparseVector,
    PointStruct,
    SearchRequest,
    SparseIndexParams,
    SparseVectorParams,
    VectorParams,
)
from transformers import AutoTokenizer

import fastembed
from fastembed.sparse.sparse_text_embedding import SparseEmbedding, SparseTextEmbedding
from fastembed.text.text_embedding import TextEmbedding

fastembed.__version__

'0.2.5'

In [3]:
dataset = load_dataset("tasksource/esci")
# We'll select the first 1000 examples for this demo
dataset = dataset["train"].select(range(1000))
dataset = dataset.filter(lambda x: x['product_locale'] == "us")
dataset

Dataset({
    features: ['example_id', 'query', 'query_id', 'product_id', 'product_locale', 'esci_label', 'small_version', 'large_version', 'product_title', 'product_description', 'product_bullet_point', 'product_brand', 'product_color', 'product_text'],
    num_rows: 919
})

## Preview Data

In [4]:
source_df = dataset.to_pandas()
df = source_df.drop_duplicates(subset=["product_text", "product_title", "product_bullet_point", "product_brand"])
df = df.dropna(subset=["product_text", "product_title", "product_bullet_point", "product_brand"])
df.head()

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,product_title,product_description,product_bullet_point,product_brand,product_color,product_text
0,0,revent 80 cfm,0,B000MOO21W,us,Irrelevant,0,1,Panasonic FV-20VQ3 WhisperCeiling 190 CFM Ceil...,None,WhisperCeiling fans feature a totally enclosed...,Panasonic,White,Panasonic FV-20VQ3 WhisperCeiling 190 CFM Ceil...
2,1,revent 80 cfm,0,B07X3Y6B1V,us,Exact,0,1,Homewerks 7141-80 Bathroom Fan Integrated LED ...,None,OUTSTANDING PERFORMANCE: This Homewerk's bath ...,Homewerks,80 CFM,Homewerks 7141-80 Bathroom Fan Integrated LED ...
3,2,revent 80 cfm,0,B07WDM7MQQ,us,Exact,0,1,Homewerks 7140-80 Bathroom Fan Ceiling Mount E...,None,OUTSTANDING PERFORMANCE: This Homewerk's bath ...,Homewerks,White,Homewerks 7140-80 Bathroom Fan Ceiling Mount E...
4,3,revent 80 cfm,0,B07RH6Z8KW,us,Exact,0,1,Delta Electronics RAD80L BreezRadiance 80 CFM ...,This pre-owned or refurbished product has been...,Quiet operation at 1.5 sones\nBuilt-in thermos...,DELTA ELECTRONICS (AMERICAS) LTD.,White,Delta Electronics RAD80L BreezRadiance 80 CFM ...
5,4,revent 80 cfm,0,B07QJ7WYFQ,us,Exact,0,1,Panasonic FV-08VRE2 Ventilation Fan with Reces...,None,The design solution for Fan/light combinations...,Panasonic,White,Panasonic FV-08VRE2 Ventilation Fan with Reces...


In [5]:
print(f"Catalog Item Count: {len(df)}\nQueries: {len(source_df)}")

Catalog Item Count: 176
Queries: 919


In [6]:
df["combined_text"] = df["product_title"] + "\n" + df["product_text"] + "\n" + df["product_bullet_point"]

In [7]:
len(df)

176

## Create Sparse Embeddings

In [8]:
sparse_model_name = "prithvida/Splade_PP_en_v1"
dense_model_name = "BAAI/bge-large-en-v1.5"
# This triggers the model download
sparse_model = SparseTextEmbedding(model_name=sparse_model_name, batch_size=32)
dense_model = TextEmbedding(model_name=dense_model_name, batch_size=32)

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

In [9]:
def make_sparse_embedding(texts: List[str]):
    return list(sparse_model.embed(texts))


sparse_embedding: List[SparseEmbedding] = make_sparse_embedding("Fastembed is a great library for text embeddings!")
sparse_embedding

[SparseEmbedding(values=array([0.17295611, 0.80484658, 0.41356239, 0.38512513, 0.90825951,
        0.61373132, 0.18313883, 0.18546289, 0.04257777, 1.20476401,
        1.48403311, 0.17008089, 0.06487759, 0.16780744, 0.23214206,
        2.5722568 , 1.87174428, 0.2541669 , 0.20749982, 0.16315481,
        0.70712435, 0.26381177, 0.49152234, 0.67282563, 0.19267203,
        0.29127747, 0.09682107, 1.21251154, 0.19741221, 0.44512141,
        0.44369081, 0.21676107, 0.36704862, 0.06706504, 1.97674787,
        0.00856015, 0.51626593, 0.21145488, 0.09790635, 0.26357391,
        1.6925323 , 2.10766435, 0.05584541, 0.05150893, 0.24062614,
        0.90479541, 0.1198509 , 0.10030396]), indices=array([ 1037,  2003,  2005,  2190,  2204,  2307,  2338,  2497,  2565,
         2793,  3075,  3177,  3274,  3286,  3430,  3435,  3793,  3819,
         3835,  3989,  4007,  4118,  4248,  4289,  4294,  4322,  4434,
         4667,  4773,  5080,  5371,  5527,  6028,  6581,  6633,  6919,
         6981,  6994,  7809,

The previous output is a SparseEmbedding object for the first document in our list.

It contains two arrays: values and indices. 
- The 'values' array represents the weights of the features (tokens) in the document.
- The 'indices' array represents the indices of these features in the model's vocabulary.

Each pair of corresponding values and indices represents a token and its weight in the document.

This is still a little abstract, so let's use the tokenizer vocab to make sense of these indices.

In [10]:
SparseTextEmbedding.list_supported_models()

[{'model': 'prithvida/Splade_PP_en_v1',
  'vocab_size': 30522,
  'description': 'Independent Implementation of SPLADE++ Model for English',
  'size_in_GB': 0.532,
  'sources': {'hf': 'Qdrant/SPLADE_PP_en_v1'}}]

In [11]:
def get_tokens_and_weights(sparse_embedding, model_name):
    # Find the tokenizer for the model
    for models in SparseTextEmbedding.list_supported_models():
        if models["model"] == model_name:
            tokenizer_source = models["sources"]["hf"]
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_source)
    token_weight_dict = {}
    for i in range(len(sparse_embedding.indices)):
        token = tokenizer.decode([sparse_embedding.indices[i]])
        weight = sparse_embedding.values[i]
        token_weight_dict[token] = weight

    # Sort the dictionary by weights
    token_weight_dict = dict(sorted(token_weight_dict.items(), key=lambda item: item[1], reverse=True))
    return token_weight_dict


# Test the function with the first SparseEmbedding
print(json.dumps(get_tokens_and_weights(sparse_embedding[0], sparse_model_name), indent=4))

{
    "fast": 2.5722568035125732,
    "##bed": 2.1076643466949463,
    "##em": 1.9767478704452515,
    "text": 1.8717442750930786,
    "em": 1.6925323009490967,
    "library": 1.4840331077575684,
    "##ding": 1.2125115394592285,
    "bed": 1.2047640085220337,
    "good": 0.9082595109939575,
    "librarian": 0.9047954082489014,
    "is": 0.8048465847969055,
    "software": 0.7071243524551392,
    "format": 0.6728256344795227,
    "great": 0.613731324672699,
    "texts": 0.5162659287452698,
    "quick": 0.49152234196662903,
    "device": 0.4451214075088501,
    "file": 0.44369080662727356,
    "for": 0.4135623872280121,
    "best": 0.38512513041496277,
    "technique": 0.36704862117767334,
    "facility": 0.2912774682044983,
    "method": 0.26381176710128784,
    "ideal": 0.26357391476631165,
    "perfect": 0.2541669011116028,
    "##bing": 0.24062614142894745,
    "material": 0.23214206099510193,
    "storage": 0.21676106750965118,
    "tool": 0.21145488321781158,
    "nice": 0.2074998

## Create Dense Embeddings

In [12]:
def make_dense_embedding(texts: List[str]):
    return list(dense_model.embed(texts))


dense_embedding = make_dense_embedding("Fastembed is a great library for text embeddings!")

In [13]:
dense_embedding[0].shape

(1024,)

In [14]:
product_texts = df["combined_text"].tolist()

In [15]:
[x for x in product_texts if not isinstance(x, str)]

[]

In [16]:
%%time
df["sparse_embedding"] = make_sparse_embedding(product_texts)

CPU times: user 4min 42s, sys: 25.2 s, total: 5min 7s
Wall time: 2min 13s


In [17]:
df["sparse_embedding"]

0      SparseEmbedding(values=array([0.06509431, 0.57...
2      SparseEmbedding(values=array([0.10595927, 0.20...
3      SparseEmbedding(values=array([0.1140037 , 0.02...
4      SparseEmbedding(values=array([6.13510251e-01, ...
5      SparseEmbedding(values=array([0.90058267, 0.12...
                             ...                        
780    SparseEmbedding(values=array([5.56782305e-01, ...
809    SparseEmbedding(values=array([0.38585788, 0.44...
828    SparseEmbedding(values=array([3.27695787e-01, ...
867    SparseEmbedding(values=array([0.36255798, 0.74...
870    SparseEmbedding(values=array([3.74321818e-01, ...
Name: sparse_embedding, Length: 176, dtype: object

In [18]:
%%time
df["dense_embedding"] = make_dense_embedding(product_texts)

CPU times: user 12min 3s, sys: 2.67 s, total: 12min 6s
Wall time: 2min 5s


## Indexing

In [19]:
client = QdrantClient(":memory:")

In [20]:
collection_name = "esci"
client.create_collection(
    collection_name,
    vectors_config={
        "text-dense": VectorParams(
            size=1024,  # OpenAI Embeddings
            distance=Distance.COSINE,
        )
    },
    sparse_vectors_config={
        "text-sparse": SparseVectorParams(
            index=SparseIndexParams(
                on_disk=False,
            )
        )
    },
)

True

In [21]:
SparseVector(indices=[1, 2, 3], values=[0.1, 0.2, 0.3])

SparseVector(indices=[1, 2, 3], values=[0.1, 0.2, 0.3])

In [22]:
def make_points(df: pd.DataFrame)-> List[PointStruct]:
    sparse_vectors = df["sparse_embedding"].tolist()
    product_texts = df["combined_text"].tolist()
    dense_vectors = df["dense_embedding"].tolist()
    points = []
    for idx, (text, sparse_vector, dense_vector) in enumerate(zip(product_texts, sparse_vectors, dense_vectors)):
        # print(sparse_vector)
        sparse_vector = SparseVector(indices=sparse_vector.indices.tolist(), values=sparse_vector.values.tolist())
        point = PointStruct(
            id=idx,
            payload={"text": text},  # Add any additional payload if necessary
            vector={
                "text-sparse": sparse_vector,
                "text-dense": dense_vector.tolist(),
            },
        )
        points.append(point)
    return points


points: List[PointStruct] = make_points(df)

In [23]:
client.upsert(collection_name, points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [24]:
query_text = "panasonic fans"

# # Compute sparse and dense vectors
query_sparse_vectors: List[SparseEmbedding] = make_sparse_embedding(query_text)
query_dense_vector: List[np.ndarray] = make_dense_embedding(query_text)

client.search_batch(
    collection_name=collection_name,
    requests=[
        SearchRequest(
            vector=NamedVector(
                name="text-dense",
                vector=query_dense_vector[0],
            ),
            limit=10,
            with_payload=True,
        ),
        SearchRequest(
            vector=NamedSparseVector(
                name="text-sparse",
                vector=SparseVector(
                    indices=query_sparse_vectors[0].indices.tolist(),
                    values=query_sparse_vectors[0].values.tolist(),
                ),
            ),
            limit=10,
            with_payload=True,
        ),
    ],
)

[[ScoredPoint(id=0, version=0, score=0.7569294929688511, payload={'text': 'Panasonic FV-20VQ3 WhisperCeiling 190 CFM Ceiling Mounted Fan\nPanasonic FV-20VQ3 WhisperCeiling 190 CFM Ceiling Mounted Fan\nPanasonic\nWhite\nNone\nWhisperCeiling fans feature a totally enclosed condenser motor and a double-tapered, dolphin-shaped bladed blower wheel to quietly move air\nDesigned to give you continuous, trouble-free operation for many years thanks in part to its high-quality components and permanently lubricated motors which wear at a slower pace\nDetachable adaptors, firmly secured duct ends, adjustable mounting brackets (up to 26-in), fan/motor units that detach easily from the housing and uncomplicated wiring all lend themselves to user-friendly installation\nThis Panasonic fan has a built-in damper to prevent backdraft, which helps to prevent outside air from coming through the fan\n0.35 amp\nWhisperCeiling fans feature a totally enclosed condenser motor and a double-tapered, dolphin-shape